In [31]:
!pip install llama-index-core llama-index-utils-workflow llama-index-llms-openai \
    llama-parse llama-index-embeddings-openai llama-index-readers-whisper gradio



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [32]:
import os, json
from llama_parse import LlamaParse
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context
)
from helper import get_openai_api_key, get_llama_cloud_api_key
from IPython.display import display, HTML
from helper import extract_html_content
from llama_index.utils.workflow import draw_all_possible_flows

In [33]:
import nest_asyncio
nest_asyncio.apply()

In [34]:
llama_cloud_api_key = get_llama_cloud_api_key()
openai_api_key = get_openai_api_key()

In [35]:
from llama_index.core.workflow import InputRequiredEvent, HumanResponseEvent

In [36]:
class ParseFormEvent(Event):
    application_form:str


class QueryEvent(Event):
    query:str
    field:str

class ResponseEvent(Event):
    response:str

class FeedbackEvent(Event):
    feedback:str

class GenerateQuestionsEvent(Event):
    pass

In [37]:
class RAGWorkflow(Workflow):

    storage_dir="./storage"
    llm= OpenAI
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx:Context , ev:StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise valueError("No Resume File provided")
        if not ev.application_form:
            raise valueError("No Application File provided")
        
        self.llm= OpenAI(model="gpt-3.5-turbo")


        if os.path.exists(self.storage_dir):

            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            
            index = load_index_from_storage(storage_context)
            
        else:
            documents=LlamaParse(
                api_key=llama_cloud_api_key,
                base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
                result_type="markdown",
                content_guideline_instruction='This is a resume, gather related facts together and format it with bullet points with headers '
            ).load_data(ev.resume_file)
            #embed and index document 

            index=VectorStoreIndex.from_documents(
                documents,
                embed_model=OpenAIEmbedding(model_name="text-embedding-3-small")
            )
            index.storage_context.persist(persist_dir=self.storage_dir)


        self.query_engine=index.as_query_engine(llm=self.llm, similarity_top_k=5)

        return ParseFormEvent(application_form)

#form Parsing
    @step
    async def parse_form(self, ctx:Context, ev: ParseFormEvent)-> GenerateQuestionsEvent:
        parser=LlamaParse(
            api_key=llama_cloud_api_key,
            base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            result_type="markdown",
            content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
            formatting_instruction="Return a bulleted list of the fields ONLY."
        )

        #get LLM to convert parsed form into JSON 
        result = parser.load_data(ev.application_form)[0]
        raw_json=self.llm.complete(
            f"This is a parsed form. Convert it into a JSON object containing only the list of fields to be filled in, in the form {{ fields: [...] }}. <form>{result.text}</form>. Return JSON ONLY, no markdown.")
        fields = json.loads(raw_json.text)["fields"]

        await ctx.set("fields_to_fill",fields)

        return GenerateQuestionsEvent()
        
    @step
    async def generate_question(self,ctx:Context, ev:GenerateQuestionsEvent | FeedbackEvent) ->QueryEvent:
#get the list of fields 
        fields =await ctx.get("fields to fill")


        for field in fields: 
            question =f"How would you answer the question about the candidate?<field>{field}</field>"


        if hasattr(ev,"feedback"):
            question += f"""
                    \n We previously got feedback about how we answered the questions.
                    It might not be relevant to this particular field, but here it is:
                    <feedback>{ev.feedback}</feedback>
                """

        ctx.send_event(QueryEvent(
            field=field,
            query=question
        ))

        await ctx.set("total_fields", len(fields))
        return

        @step
        async def ask_question(self, ctx:Context, ev:QueryEvent)-> ResponseEvent:
            response=self.query_engine.query(f"this is a question about specific resume in our database:{ev.query}")
            return ResponseEvent (field=ev.field , response=response.response)


        # Get feedback from the human
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> InputRequiredEvent:
        # get the total number of fields to wait for
        total_fields = await ctx.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return None # do nothing if there's nothing to do yet

        # we've got all the responses!
        responseList = "\n".join("Field: " + r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseList}
            </responses>
        """)

        # save the result for later
        await ctx.set("filled_form", str(result))

        # Fire off the feedback request
        return InputRequiredEvent(
            prefix="How does this look? Give me any feedback you have on any of the answers.",
            result=result
        )


        @step
        # async def get_feedback(self, ctx:Context, ev: HumanResponseEvent)-> FeedbackEvent | StopEvent:
        #         result=self.llm.complete(f"""
        #     You have received some human feedback on the form-filling task you've done.
        #     Does everything look good, or is there more work to be done?
        #     <feedback>
        #     {ev.response}
        #     </feedback>
        #     If everything is fine, respond with just the word 'OKAY'.
        #     If there's any other feedback, respond with just the word 'FEEDBACK'.
        # """)

        # verdict= result.text.strip()
        # print(f"LLM says the verdict was {verdict}")
        # if (verdict =="OKAY"):
        #     return StopEvent(result=await ctx.get("filed_form"))
        # else:
        #     return FeedBackEvent(feedback=ev.repsonse)

        async def get_feedback(self, ctx: Context, ev: HumanResponseEvent) -> FeedbackEvent | StopEvent:

          result = self.llm.complete(f"""
            You have received some human feedback on the form-filling task you've done.
            Does everything look good, or is there more work to be done?
            <feedback>
            {ev.response}
            </feedback>
            If everything is fine, respond with just the word 'OKAY'.
            If there's any other feedback, respond with just the word 'FEEDBACK'.
        """)

        verdict = result.text.strip()

        print(f"LLM says the verdict was {verdict}")
        if (verdict == "OKAY"):
            return StopEvent(result=await ctx.get("filled_form"))
        else:
            return FeedbackEvent(feedback=ev.response)




In [38]:
class RAGWorkflow(Workflow):

    storage_dir = "./storage"
    llm: OpenAI
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        self.llm = OpenAI(model="gpt-3.5-turbo")

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=
                                                           self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse and load the resume document
            documents = LlamaParse(
                api_key=llama_cloud_api_key,
                base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
                result_type="markdown",
                content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
            ).load_data(ev.resume_file)
            
# Debugging: Print parsed resume text
            print("✅ Parsed Resume Text:", documents[0].text if documents else "❌ No text extracted from resume")
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=OpenAIEmbedding(model_name="text-embedding-3-small")
            )
            index.storage_context.persist(persist_dir=self.storage_dir)
            print(f"✅ Indexed {len(index.docstore.docs)} documents")

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    # form parsing
    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> GenerateQuestionsEvent:
        parser = LlamaParse(
            api_key=llama_cloud_api_key,
            base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            result_type="markdown",
            content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
            formatting_instruction="Return a bulleted list of the fields ONLY."
        )
        # data = parser.load_data(ev.application_form)
        # print("Parsed Data:", data)  # ✅ Debugging line to check if data exists

        # if not data:
        #     raise ValueError("Error: LlamaParse returned an empty list. Please check if the application form is valid.")
        # result = data[0]  # This will now only execute if data is not empty

        # get the LLM to convert the parsed form into JSON
        result = parser.load_data(ev.application_form)[0]
        raw_json = self.llm.complete(
            f"This is a parsed form. Convert it into a JSON object containing only the list of fields to be filled in, in the form {{ fields: [...] }}. <form>{result.text}</form>. Return JSON ONLY, no markdown.")
        fields = json.loads(raw_json.text)["fields"]

        await ctx.set("fields_to_fill", fields)

        return GenerateQuestionsEvent()

    # generate questions
    @step
    async def generate_questions(self, ctx: Context, ev: GenerateQuestionsEvent | FeedbackEvent) -> QueryEvent:

        # get the list of fields to fill in
        fields = await ctx.get("fields_to_fill")

        # generate one query for each of the fields, and fire them off
        for field in fields:
            question = f"How would you answer this question about the candidate? <field>{field}</field>"

            # new! Is there feedback? If so, add it to the query:
            if hasattr(ev,"feedback"):
                question += f"""
                    \nWe previously got feedback about how we answered the questions.
                    It might not be relevant to this particular field, but here it is:
                    <feedback>{ev.feedback}</feedback>
                """

            ctx.send_event(QueryEvent(
                field=field,
                query=question
            ))

        # store the number of fields so we know how many to wait for later
        await ctx.set("total_fields", len(fields))
        return

    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return ResponseEvent(field=ev.field, response=response.response)


    # Get feedback from the human
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> InputRequiredEvent:
        # get the total number of fields to wait for
        total_fields = await ctx.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return None # do nothing if there's nothing to do yet

        # we've got all the responses!
        responseList = "\n".join("Field: " + r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseList}
            </responses>
        """)

        # save the result for later
        await ctx.set("filled_form", str(result))

        # Fire off the feedback request
        return InputRequiredEvent(
            prefix="How does this look? Give me any feedback you have on any of the answers.",
            result=result
        )

    # Accept the feedback when a HumanResponseEvent fires
    @step
    async def get_feedback(self, ctx: Context, ev: HumanResponseEvent) -> FeedbackEvent | StopEvent:

        result = self.llm.complete(f"""
            You have received some human feedback on the form-filling task you've done.
            Does everything look good, or is there more work to be done?
            <feedback>
            {ev.response}
            </feedback>
            If everything is fine, respond with just the word 'OKAY'.
            If there's any other feedback, respond with just the word 'FEEDBACK'.
        """)

        verdict = result.text.strip()

        print(f"LLM says the verdict was {verdict}")
        if (verdict == "OKAY"):
            return StopEvent(result=await ctx.get("filled_form"))
        else:
            return FeedbackEvent(feedback=ev.response)


In [45]:
w = RAGWorkflow (timeout=600 , verbose=False)

handler =w.run(
    resume_file='/Users/saulehdeshmukh/Desktop/EventDrivenAgenticDocumentWorkflow/fake_resume.pdf',
   application_form='/Users/saulehdeshmukh/Desktop/EventDrivenAgenticDocumentWorkflow/fake_application_form.pdf'
    
    )

async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        print("We've filled in your form! Here are the results:\n")
        print(event.result)
        # now ask for input from the keyboard
        response = input(event.prefix)
        handler.ctx.send_event(
            HumanResponseEvent(
                response=response
            )
        )

response = await handler
print("Agent complete! Here's your final result:")
print(str(response))

Started parsing the file under job_id 44cd4317-fe25-4f23-9fee-57e38655bcf1
We've filled in your form! Here are the results:

<filled_fields>
First Name: [No response provided]
Last Name: [No response provided]
Email: [No response provided]
Phone: [No response provided]
Linkedin: [No response provided]
Project Portfolio: [No response provided]
Degree: [No response provided]
Graduation Date: [No response provided]
Current Job Title: [No response provided]
Current Employer: [No response provided]
Technical Skills: [No response provided]
Describe why you’re a good fit for this position: [No response provided]
Do you have 5 years of experience in React?: [No response provided]
</filled_fields>


How does this look? Give me any feedback you have on any of the answers. ok


LLM says the verdict was OKAY
Agent complete! Here's your final result:
<filled_fields>
First Name: [No response provided]
Last Name: [No response provided]
Email: [No response provided]
Phone: [No response provided]
Linkedin: [No response provided]
Project Portfolio: [No response provided]
Degree: [No response provided]
Graduation Date: [No response provided]
Current Job Title: [No response provided]
Current Employer: [No response provided]
Technical Skills: [No response provided]
Describe why you’re a good fit for this position: [No response provided]
Do you have 5 years of experience in React?: [No response provided]
</filled_fields>


In [47]:
print(f"📄 Parsed resume:\n{documents[0].text[:1000]}")


NameError: name 'documents' is not defined

In [41]:
print(f"✅ Resume file path from event: {ev.resume_file}")


NameError: name 'ev' is not defined

In [69]:
class RAGWorkflow(Workflow):
    
    storage_dir = "./storage"
    llm: OpenAI
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        self.llm = OpenAI(model="gpt-3.5-turbo")

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=
                                                           self.storage_dir)
            index = load_index_from_storage(storage_context)
        # else:
            # # parse and load the resume document
            # documents = LlamaParse(
            #     api_key=llama_cloud_api_key,
            #     base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            #     result_type="markdown",
            #     content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
            # ).load_data(ev.resume_file)
            # # embed and index the documents
            # index = VectorStoreIndex.from_documents(
            #     documents,
            #     embed_model=OpenAIEmbedding(model_name="text-embedding-3-small")
            # )
            # index.storage_context.persist(persist_dir=self.storage_dir)

        else:
    # parse and load the resume document
            documents = LlamaParse(
            api_key=llama_cloud_api_key,
            base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            result_type="markdown",
            content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
            ).load_data(ev.resume_file)
    
            print("✅ Parsed Resume Sample:\n", documents[0].text[:1000])  # 👈 Put it here
        
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=OpenAIEmbedding(model_name="text-embedding-3-small")
            )
            index.storage_context.persist(persist_dir=self.storage_dir)
    

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    # form parsing
    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> GenerateQuestionsEvent:
        parser = LlamaParse(
            api_key=llama_cloud_api_key,
            base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            result_type="markdown",
            content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
            formatting_instruction="Return a bulleted list of the fields ONLY."
        )

        # get the LLM to convert the parsed form into JSON
        result = parser.load_data(ev.application_form)[0]
        raw_json = self.llm.complete(
            f"This is a parsed form. Convert it into a JSON object containing only the list of fields to be filled in, in the form {{ fields: [...] }}. <form>{result.text}</form>. Return JSON ONLY, no markdown.")
        fields = json.loads(raw_json.text)["fields"]

        await ctx.set("fields_to_fill", fields)

        return GenerateQuestionsEvent()

    # generate questions
    @step
    async def generate_questions(self, ctx: Context, ev: GenerateQuestionsEvent | FeedbackEvent) -> QueryEvent:

        # get the list of fields to fill in
        fields = await ctx.get("fields_to_fill")

        # generate one query for each of the fields, and fire them off
        for field in fields:
            question = f"How would you answer this question about the candidate? <field>{field}</field>"

            # new! Is there feedback? If so, add it to the query:
            if hasattr(ev,"feedback"):
                question += f"""
                    \nWe previously got feedback about how we answered the questions.
                    It might not be relevant to this particular field, but here it is:
                    <feedback>{ev.feedback}</feedback>
                """
            
            ctx.send_event(QueryEvent(
                field=field,
                query=question
            ))

        # store the number of fields so we know how many to wait for later
        await ctx.set("total_fields", len(fields))
        return
        
    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return ResponseEvent(field=ev.field, response=response.response)

  
    # Get feedback from the human
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> InputRequiredEvent:
        # get the total number of fields to wait for
        total_fields = await ctx.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return None # do nothing if there's nothing to do yet

        # we've got all the responses!
        responseList = "\n".join("Field: " + r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseList}
            </responses>
        """)

        # save the result for later
        await ctx.set("filled_form", str(result))

        # Fire off the feedback request
        return InputRequiredEvent(
            prefix="How does this look? Give me any feedback you have on any of the answers.",
            result=result
        )

    # Accept the feedback when a HumanResponseEvent fires
    @step
    async def get_feedback(self, ctx: Context, ev: HumanResponseEvent) -> FeedbackEvent | StopEvent:

        result = self.llm.complete(f"""
            You have received some human feedback on the form-filling task you've done.
            Does everything look good, or is there more work to be done?
            <feedback>
            {ev.response}
            </feedback>
            If everything is fine, respond with just the word 'OKAY'.
            If there's any other feedback, respond with just the word 'FEEDBACK'.
        """)

        verdict = result.text.strip()

        print(f"LLM says the verdict was {verdict}")
        if (verdict == "OKAY"):
            return StopEvent(result=await ctx.get("filled_form"))
        else:
            return FeedbackEvent(feedback=ev.response)


In [71]:
w = RAGWorkflow (timeout=600 , verbose=False)

handler =w.run(
    resume_file='/Users/saulehdeshmukh/Desktop/EventDrivenAgenticDocumentWorkflow/fake_resume.pdf',
   application_form='/Users/saulehdeshmukh/Desktop/EventDrivenAgenticDocumentWorkflow/fake_application_form.pdf'
    
    )

async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        print("We've filled in your form! Here are the results:\n")
        print(event.result)
        # now ask for input from the keyboard
        response = input(event.prefix)
        handler.ctx.send_event(
            HumanResponseEvent(
                response=response
            )
        )

response = await handler
print("Agent complete! Here's your final result:")
print(str(response))

Started parsing the file under job_id 00bc9510-4619-4ff7-9d3e-e0cb4390c197
✅ Parsed Resume Sample:
 # Sarah Chen

Email: sarah.chen@email.com

LinkedIn: linkedin.com/in/sarahchen

GitHub: github.com/sarahcodes

Portfolio: sarahchen.dev

Location: San Francisco, CA

# Professional Summary

Innovative Full Stack Web Developer with 6+ years of experience crafting scalable web applications and microservices. Specialized in React, Node.js, and cloud architecture. Proven track record of leading technical teams and implementing CI/CD pipelines that reduced deployment time by 40%. Passionate about clean code, accessibility, and mentoring junior developers.

# Professional Experience

# Senior Full Stack Developer

TechFlow Solutions | San Francisco, CA January 2022 - Present

- Architected and implemented a microservices-based e-commerce platform serving 100K+ daily users
- Led a team of 5 developers in rebuilding the company's flagship product using React and Node.js
- Implemented GraphQL API

How does this look? Give me any feedback you have on any of the answers. ok


LLM says the verdict was OKAY
Agent complete! Here's your final result:
<Field: First Name
Response: Sarah
Field: Last Name
Response: Chen
Field: Email
Response: sarah.chen@email.com
Field: Phone
Response: Not provided
Field: Linkedin
Response: linkedin.com/in/sarahchen
Field: Project Portfolio
Response: The candidate's project portfolio includes developing a carbon footprint tracking application called EcoTrack using React, Node.js, and MongoDB, which also featured a machine learning algorithm for personalized sustainability recommendations. Another project, ChatFlow, involved creating a real-time chat application with WebSocket protocol and React, serving over 5000 monthly active users. The candidate's projects showcase a range of technical skills and experiences in building web applications and services.
Field: Degree
Response: Bachelor of Science in Computer Science
Field: Graduation Date
Response: 2017
Field: Current Job Title
Response: Senior Full Stack Developer
Field: Current E